In [ ]:
import numpy as np
import os
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from numpy import dot
from numpy.linalg import norm


from collections import Counter
import matplotlib.pyplot as plt
!unzip "/content/cora (extract.me).zip"


Archive:  /content/cora (extract.me).zip
replace cora/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cora/README             
replace cora/cora.cites? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cora/cora.cites         
replace cora/cora.content? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cora/cora.content       


In [ ]:
# from sklearn.utils import shuffle

In [ ]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

                
# random_state = 42
# all_data = shuffle(all_data,random_state=random_state)

In [ ]:
categories =  ['Reinforcement_Learning', 'Theory', 'Case_Based', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Rule_Learning']
sorted(categories)
label_encoder = {}
i = 0
for cat in sorted(categories):
  label_encoder[cat] = i
  i +=1
label_encoder


{'Case_Based': 0,
 'Genetic_Algorithms': 1,
 'Neural_Networks': 2,
 'Probabilistic_Methods': 3,
 'Reinforcement_Learning': 4,
 'Rule_Learning': 5,
 'Theory': 6}

In [ ]:
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(label_encoder[elements[-1]])
    X.append(elements[1:-1])
    nodes.append(elements[0])
    element_to_ind[elements[0]]= i
X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)


X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {0, 1, 2, 3, 4, 5, 6}

Number of classes:  7


In [ ]:
!pip install --upgrade networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
print('Graph info: Graph with ', len(G.nodes), " nodes and ", len(G.edges), " edges")

Graph info: Graph with  2708  nodes and  5278  edges


In [ ]:
nodes = list(G.nodes)
print(len(nodes))
list(G.neighbors(0))

2708


[469, 683, 2160, 2238, 1571]

In [ ]:
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
df.head()

2708


node  label                                           features
0     0      2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     1      2  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...
2     2      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     3      6  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     4      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

In [ ]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)

In [ ]:
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes 
df.head()

<ipython-input-11-2046024a68d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['node'] = list(range(len(df))) #rename nodes


node  label                                           features
0     0      2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     1      2  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...
2     2      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     3      6  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     4      1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

In [ ]:
train = df.groupby('label', group_keys=False).apply(lambda x: x.sample(20))
G = nx.relabel_nodes(G, df['node'])

In [ ]:
def create_transition_matrix(g):
    vs = list(g.nodes)
    n = len(vs)
    adj = nx.adjacency_matrix(g).toarray()
    transition_matrix = adj/adj.sum(axis=1)

    return transition_matrix

In [ ]:
def random_walk(g, num_steps, start_node, transition_matrix=None):
    if transition_matrix is None:
        transition_matrix = create_transition_matrix(g)
    node = start_node
    v = list(node)
    for i in range(num_steps):
        neighbors = list(g.neighbors(node))
        # get probability of transition
        p = transition_matrix[node, neighbors]
        if p.sum()!= 0:
            p /= p.sum()  
        node1=np.random.choice(neighbors,p=p)
        v.append(node1)
        node = node1
    return v

In [ ]:
seeds_dict = {predicted:list(train[train['label'] == predicted]['node']) for predicted in range(7)}

def random_walk_with_teleportation(g, num_steps, start_node,tp,predicted, transition_matrix = None):
    #modify random walk code to add teleportation.
    #you can only teleport to a node belonging to the same class as the start node
    if transition_matrix is None:
        transition_matrix = create_transition_matrix(g)
    node = start_node
    v = list(node)
    for k in range(num_steps):
        neighbors = list(g.neighbors(node))
        p = np.asarray(transition_matrix[node, neighbors])[0]
        # apply teleportation
        if np.random.random() < tp:
            t = seeds_dict[predicted]
            node1 = np.random.choice(t)
        else:
            node1 = np.random.choice(neighbors, p=p)
        v.append(node1)
        node = node1
    return v

In [ ]:
#pagerank. NO teleportation, NO tfidf. 
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

for train_node, predicted in zip(train['node'], train['label']):
  for i in range(num_samples):
      start_point = train_node
      walk = random_walk(G, num_walk_steps, start_point, transition_matrix)
      end_node = walk[-1] 
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] += 1


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

unvisited =  1
              precision    recall  f1-score   support

           0       0.65      0.72      0.68       285
           1       0.80      0.90      0.85       406
           2       0.82      0.53      0.64       726
           3       0.76      0.68      0.72       379
           4       0.43      0.57      0.49       214
           5       0.29      0.77      0.42       131
           6       0.62      0.49      0.55       344

    accuracy                           0.65      2485
   macro avg       0.62      0.67      0.62      2485
weighted avg       0.70      0.65      0.65      2485



0.6454728370221328

In [ ]:
#pagerank. WITH telportation, without tfidf 
transition_matrix = create_transition_matrix(G)
num_samples = 1000  
num_walk_steps = 100
#vary teleportation probability in (0, 0.1)
tp = [0, 0.1, 0.2]
visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]
for j in tp:
    print("tp: ", j)
    for train_node, predicted in zip(train['node'], train['label']):
        for i in range(num_samples):
            start_point = train_node
            walk = random_walk_with_teleportation(G, num_walk_steps, start_point, j, predicted, transition_matrix)
            end_node = walk[-1]  
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] += 1
    #get metrics
    count = 0 #these many nodes remain unvisited. 
    for vf in visiting_freq:
      if vf ==0:
        count+=1
    print('unvisited = ', count)
    visiting_freq_label = np.asarray(visiting_freq_label)
    preds = np.argmax(visiting_freq_label,axis = 1)
    print(classification_report(df['label'], preds))
    print(accuracy_score(df['label'], preds))

C:\Users\Yuanxuan\AppData\Local\Temp\ipykernel_14064\2140906970.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(g)


tp:  0
unvisited =  0
              precision    recall  f1-score   support

           0       0.29      0.48      0.36       285
           1       0.49      0.57      0.53       406
           2       0.64      0.33      0.43       726
           3       0.67      0.58      0.62       379
           4       0.13      0.17      0.15       214
           5       0.09      0.18      0.12       131
           6       0.33      0.27      0.30       344

    accuracy                           0.39      2485
   macro avg       0.38      0.37      0.36      2485
weighted avg       0.46      0.39      0.41      2485

0.39235412474849096
tp:  0.1
unvisited =  0
              precision    recall  f1-score   support

           0       0.66      0.81      0.73       285
           1       0.81      0.91      0.86       406
           2       0.86      0.53      0.65       726
           3       0.79      0.70      0.74       379
           4       0.50      0.80      0.62       214
           5

In [ ]:
vs = list(G.nodes)
n = len(vs)
adj = nx.adjacency_matrix(G)
transition = np.zeros((len(G.nodes), len(G.nodes)))

#for n1 in nodes:
  #for n2 in nodes:
    # if there is an edge between n1 and n2:
      # cos_sim = compute cosine similarity between features of n1 and n2
      # transition[n1,n2] = np.exp(cos_sim) #neumerator of softmax. #why do we need softmax?  
#divide the values in transition by denominator of softmax. how will you do this? 


In [ ]:
#pagerank. Without teleportation. WITH TFIDF 
transition_matrix = transition

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

for train_node, predicted in zip(train['node'], train['label']):
    for i in range(num_samples):
        start_point = train_node
        walk = random_walk(G, num_walk_steps, start_point, transition_matrix)
        end_node = walk[-1] 
        visiting_freq_label[end_node][predicted] += 1
        visiting_freq[end_node] += 1
#perfrom pagerank using our tf_idf based transition matrix
#use randon walk without teleporation 
#get metrics


In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

unvisited =  0
              precision    recall  f1-score   support

           0       0.31      0.51      0.39       285
           1       0.48      0.56      0.52       406
           2       0.50      0.26      0.34       726
           3       0.62      0.51      0.56       379
           4       0.09      0.13      0.11       214
           5       0.12      0.27      0.17       131
           6       0.30      0.23      0.26       344

    accuracy                           0.36      2485
   macro avg       0.35      0.35      0.33      2485
weighted avg       0.41      0.36      0.37      2485



0.3609657947686117

In [ ]:
#pagerank. WITH teleportation WITH TFIDF
transition_matrix = transition

num_samples = 1000  
num_walk_steps = 100
#vary teleportation probability in (0, 0.1)
tp = [0, 0.1, 0.2]
visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]
for j in tp:
    print("tp: ", j)
    for train_node, predicted in zip(train['node'], train['label']):
        for i in range(num_samples):
            start_point = train_node
            walk = random_walk_with_teleportation(G, num_walk_steps, start_point, j, predicted, transition_matrix)
            end_node = walk[-1]  # Get the last node in the random walk
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] += 1
    #get metrics
    count = 0 #these many nodes remain unvisited. 
    for vf in visiting_freq:
      if vf ==0:
        count+=1
    print('unvisited = ', count)
    visiting_freq_label = np.asarray(visiting_freq_label)
    preds = np.argmax(visiting_freq_label,axis = 1)
    print(classification_report(df['label'], preds))
    print(accuracy_score(df['label'], preds))

tp:  0
unvisited =  0
              precision    recall  f1-score   support

           0       0.32      0.49      0.39       285
           1       0.49      0.59      0.53       406
           2       0.62      0.33      0.43       726
           3       0.70      0.55      0.62       379
           4       0.13      0.17      0.15       214
           5       0.11      0.25      0.15       131
           6       0.30      0.25      0.27       344

    accuracy                           0.39      2485
   macro avg       0.38      0.38      0.36      2485
weighted avg       0.46      0.39      0.41      2485

0.39436619718309857
tp:  0.1
unvisited =  0
              precision    recall  f1-score   support

           0       0.68      0.81      0.74       285
           1       0.78      0.92      0.84       406
           2       0.87      0.54      0.67       726
           3       0.80      0.70      0.74       379
           4       0.53      0.82      0.64       214
           5